In [1]:
import serial
import time

class ElliptecBus:
    """Single serial connection controlling multiple Elliptec devices on the same bus."""
    def __init__(self, port='COM3', baudrate=9600, verbose=False):
        self.port = port
        self.baudrate = baudrate
        self.verbose = verbose
        
        self.ser = serial.Serial(
            port=self.port,
            baudrate=self.baudrate,
            bytesize=serial.EIGHTBITS,
            stopbits=serial.STOPBITS_ONE,
            parity=serial.PARITY_NONE,
            timeout=2
        )
        time.sleep(0.1)
        self._purge()

    def _purge(self):
        self.ser.reset_input_buffer()
        self.ser.reset_output_buffer()
        time.sleep(0.05)

    def send(self, address, cmd):
        full = f"{address}{cmd}"
        self.ser.write(full.encode('ascii'))
        self.ser.write(b"\r\n")
        resp = self.ser.readline().decode('ascii', errors='ignore').strip()
        return resp

    def close(self):
        self.ser.close()


class ElliptecDevice:
    """Represents a single device (rotation mount) on the Elliptec bus."""
    def __init__(self, bus: ElliptecBus, address='0', verbose=False):
        self.bus = bus
        self.address = address  # hex address
        self.verbose = verbose

        self.pulses_per_turn = 143360
        self.position = 0.0

    def _send_cmd(self, cmd):
        resp = self.bus.send(self.address, cmd)
        return resp

    def _deg_to_pulses(self, degrees):
        return int(degrees * self.pulses_per_turn / 360)

    def _pulses_to_deg(self, pulses):
        if pulses >= 0x80000000:
            pulses -= 0x100000000
        return pulses * 360.0 / self.pulses_per_turn

    def _update_position(self, resp):
        try:
            self.position = round(self._pulses_to_deg(int(resp[-8:], 16)), 2)
        except:
            pass

    def get_info(self):
        return self._send_cmd("in")

    def get_status(self):
        return self._send_cmd("gs")

    def get_position_deg(self):
        return self.position

    def home(self, direction=0):
        resp = self._send_cmd(f"ho{direction}")
        self._update_position(resp)
        if self.verbose:
            print(f"Homed device {self.address}. Position: {self.position}°")
        return resp

    def move_absolute(self, pulses):
        hexval = format(pulses & 0xFFFFFFFF, '08X')
        resp = self._send_cmd(f"ma{hexval}")
        self._update_position(resp)
        return resp

    def move_relative(self, pulses):
        hexval = format(pulses & 0xFFFFFFFF, '08X')
        resp = self._send_cmd(f"mr{hexval}")
        self._update_position(resp)
        return resp

    def move_absolute_deg(self, degrees):
        pulses = self._deg_to_pulses(degrees)
        return self.move_absolute(pulses)

    def move_relative_deg(self, degrees):
        pulses = self._deg_to_pulses(degrees)
        return self.move_relative(pulses)

    def stop(self):
        resp = self._send_cmd("st")
        self._update_position(resp)
        return resp

In [2]:
bus = ElliptecBus("COM3")
rot0 = ElliptecDevice(bus, "0")
rot2 = ElliptecDevice(bus, "2")

In [5]:
# Example usage:
print("Device 0 info:", rot0.get_info())
print("Device 2 info:", rot2.get_info())

print("Status 0:", rot0.get_status())
print("Status 2:", rot2.get_status())

rot0.home()
rot2.home()

rot0.move_absolute_deg(90)
rot2.move_absolute_deg(180)

rot0.move_relative_deg(-45)
rot2.move_relative_deg(30)

print("Position 0:", rot0.get_position_deg())
print("Position 2:", rot2.get_position_deg())


Device 0 info: 0IN0E1140132020251801016800023000
Device 2 info: 2IN0E1140134020251801016800023000
Status 0: 0GS00
Status 2: 2GS00
Position 0: 45.0
Position 2: 210.0
